
# Multi-currency market making (FX) — closed-form approximation + Riccati ODE

This notebook implements the scalable approximation described in:

**Alexander Barzykin, Philippe Bergault, Olivier Guéant (2023)**  
*Dealing with multi-currency inventory risk in FX cash markets* (arXiv:2207.04100v4).

The key steps implemented below are:

1. **Logistic client-demand model** (intensity) and the associated **Hamiltonian**  
   \(H^{n,i,j}(z,p)=\sup_\delta f^{n,i,j}(z,\delta)(\delta-p)\).
2. **Quadratic approximation** of the client Hamiltonians around \(p=0\):  
   \(\check H(z,p)=\alpha_0(z)+\alpha_1(z)p+\tfrac12\alpha_2(z)p^2\).
3. Construction of the low-dimensional objects \(M,\tilde M,P\) and the **matrix Riccati-like ODE** for \(A(t)\) and \(B(t)\) (Eq. (5) in the paper).
4. Approximate quotes \(\check\delta\) (Eq. (2) with \(\theta\to\check\theta\)) and hedging rates \(\check\xi\) (Eq. (3) with \(\theta\to\check\theta\), as written on p.6 of the paper).

> **Design goals**
> - All model parameters live in one config section.
> - The main routine is wrapped as `run_multicurrency_mm(...)`.
> - Logic is split into small, testable functions.


In [ ]:

# --- Imports ---
from __future__ import annotations

from dataclasses import dataclass, field
from typing import Dict, List, Tuple, Optional, Callable

import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=6, suppress=True)

BP = 1e-4          # 1 basis point in decimal
DAY_SECONDS = 86400.0



## 1) Parameters / configuration

To experiment with **fewer currencies**, either:
- build `ModelParams` with a smaller `currencies` list, and include only the matching pairs in `pairs`, **or**
- build the full model, then call `restrict_currencies(...)`.

The example builder `build_paper_example_params()` matches **Table 1** of the paper (p.7) for:
USD, EUR, JPY, GBP, CHF with 2 tiers and size ladder \(z=\{1,5,10,20,50\}\) M$.


In [ ]:

@dataclass(frozen=True)
class TierParams:
    # Logistic f(z,δ) = 1/(1+exp(alpha + beta*δ))
    # NOTE: beta is in 1/decimal (not 1/bps). If you have beta in 1/bps, multiply by 1e4.
    alpha: float
    beta: float

@dataclass(frozen=True)
class PairParams:
    # Unordered pair (ccyA, ccyB) with canonical sorting
    pair: Tuple[str, str]

    # Client sizes (in M$ of reference currency) and baseline intensities λ(z) (per day)
    sizes_musd: np.ndarray
    lambdas_per_day: np.ndarray

    # Client tiers
    tiers: List[TierParams]

    # Hedging execution costs on the D2D segment:
    # L(ξ) = psi*|ξ| + eta*|ξ|^2, with ξ in (M$/day)
    psi: float  # decimal (e.g. 0.1 bps -> 1e-5)
    eta: float  # decimal*day/M$

@dataclass
class ModelParams:
    currencies: List[str]
    ref_ccy: str = "USD"

    # vol per currency in decimal / sqrt(day). (Ref currency vol is 0)
    sigma: Dict[str, float] = field(default_factory=dict)

    # correlation between non-ref currencies (keys are sorted tuples)
    corr: Dict[Tuple[str, str], float] = field(default_factory=dict)

    # permanent market impact per currency k_i in decimal / M$
    k: Dict[str, float] = field(default_factory=dict)

    # drift per currency μ_i in decimal / day (can be 0)
    mu: Dict[str, float] = field(default_factory=dict)

    # risk aversion γ in 1/M$
    gamma: float = 20.0

    # terminal penalty κ (dxd). If None -> zeros.
    kappa: Optional[np.ndarray] = None

    # horizon in days
    T_days: float = 0.05

    # currency-pair parameters (keys are sorted tuples)
    pairs: Dict[Tuple[str, str], PairParams] = field(default_factory=dict)


def _canon_pair(a: str, b: str) -> Tuple[str, str]:
    return tuple(sorted((a, b)))


def build_paper_example_params() -> ModelParams:
    """Reproduce the paper's 5-currency numerical example (Table 1, p.7)."""
    currencies = ["USD", "EUR", "JPY", "GBP", "CHF"]

    # --- vol in sqrt(bps/day) from Table 1 -> convert to decimal/sqrt(day) ---
    sigma = {
        "USD": 0.0,
        "EUR": 80.0 * BP,
        "GBP": 70.0 * BP,
        "CHF": 60.0 * BP,
        "JPY": 60.0 * BP,
    }

    # --- correlations between USD-based legs (Table 1, 'Crosses' block) ---
    corr_raw = {
        ("EUR", "GBP"): 0.6,
        ("EUR", "CHF"): 0.5,
        ("EUR", "JPY"): 0.3,
        ("GBP", "CHF"): 0.3,
        ("GBP", "JPY"): 0.2,
        ("CHF", "JPY"): 0.4,
    }
    corr = {tuple(sorted(k)): v for k, v in corr_raw.items()}

    # --- market impact k in bps/M$ from Table 1 (direct pairs) -> decimal/M$ ---
    k = {
        "USD": 0.0,
        "EUR": 5e-3 * BP,
        "GBP": 7e-3 * BP,
        "CHF": 8e-3 * BP,
        "JPY": 6e-3 * BP,
    }

    # drift is assumed 0 in the paper's numerical section
    mu = {c: 0.0 for c in currencies}

    sizes = np.array([1.0, 5.0, 10.0, 20.0, 50.0])  # M$

    # Table 1 (p.7). beta is listed in 1/bps, so we convert: beta_decimal = beta_bps * 1e4
    # psi listed in bps -> decimal; eta listed in (bps*day)/M$ -> (decimal*day)/M$.
    table = {
        ("EUR", "USD"): dict(lambdas=[900, 540, 234, 90, 36], alpha=[-1.9, -0.3], beta=[11, 3.5], psi_bps=0.1, eta_bps=1e-5),
        ("GBP", "USD"): dict(lambdas=[600, 200, 150, 40, 10], alpha=[-1.4,  0.0], beta=[5.5, 2.0], psi_bps=0.15, eta_bps=1.5e-5),
        ("CHF", "USD"): dict(lambdas=[420, 140, 105, 28,  7], alpha=[-1.2,  0.0], beta=[4.5, 1.9], psi_bps=0.25, eta_bps=2.5e-5),
        ("JPY", "USD"): dict(lambdas=[825, 375, 180,105, 15], alpha=[-1.6, -0.1], beta=[9.0, 3.0], psi_bps=0.1, eta_bps=1.5e-5),

        ("EUR", "GBP"): dict(lambdas=[400, 50, 25, 20, 5], alpha=[-0.5, 0.5], beta=[3.5, 2.5], psi_bps=0.25, eta_bps=3e-5),
        ("EUR", "CHF"): dict(lambdas=[400, 50, 25, 20, 5], alpha=[-0.5, 0.5], beta=[3.5, 2.5], psi_bps=0.25, eta_bps=3e-5),
        ("EUR", "JPY"): dict(lambdas=[400, 50, 25, 20, 5], alpha=[-0.5, 0.5], beta=[3.5, 2.5], psi_bps=0.25, eta_bps=3e-5),

        ("GBP", "CHF"): dict(lambdas=[160, 20, 10,  8, 2], alpha=[-0.5, 0.5], beta=[3.5, 2.5], psi_bps=0.4, eta_bps=5e-5),
        ("GBP", "JPY"): dict(lambdas=[160, 20, 10,  8, 2], alpha=[-0.5, 0.5], beta=[3.5, 2.5], psi_bps=0.4, eta_bps=5e-5),
        ("CHF", "JPY"): dict(lambdas=[ 80, 10,  5,  4, 1], alpha=[-0.5, 0.5], beta=[3.5, 2.5], psi_bps=0.4, eta_bps=5e-5),
    }

    pairs: Dict[Tuple[str, str], PairParams] = {}
    for (a, b), p in table.items():
        tiers = [
            TierParams(alpha=p["alpha"][0], beta=p["beta"][0] * 1e4),
            TierParams(alpha=p["alpha"][1], beta=p["beta"][1] * 1e4),
        ]
        key = _canon_pair(a, b)
        pairs[key] = PairParams(
            pair=key,
            sizes_musd=sizes,
            lambdas_per_day=np.array(p["lambdas"], dtype=float),
            tiers=tiers,
            psi=p["psi_bps"] * BP,
            eta=p["eta_bps"] * BP,
        )

    d = len(currencies)
    kappa = np.zeros((d, d))  # paper uses κ=0 in numerical section

    return ModelParams(
        currencies=currencies,
        ref_ccy="USD",
        sigma=sigma,
        corr=corr,
        k=k,
        mu=mu,
        gamma=20.0,
        kappa=kappa,
        T_days=0.05,
        pairs=pairs,
    )


def restrict_currencies(mp: ModelParams, keep: List[str]) -> ModelParams:
    """Return a new ModelParams restricted to a subset of currencies, keeping matching pairs."""
    keep_set = set(keep)
    if mp.ref_ccy not in keep_set:
        raise ValueError(f\"ref_ccy={mp.ref_ccy} must be included in keep={keep}\")
    new_currencies = [c for c in mp.currencies if c in keep_set]

    # Filter pairs
    new_pairs = {
        k: v for k, v in mp.pairs.items()
        if (k[0] in keep_set and k[1] in keep_set)
    }

    # Filter per-currency dicts
    def _filt(dct):
        return {k: v for k, v in dct.items() if k in keep_set}

    # Filter correlations
    new_corr = {
        k: v for k, v in mp.corr.items()
        if (k[0] in keep_set and k[1] in keep_set)
    }

    d = len(new_currencies)
    kappa = mp.kappa
    if kappa is None:
        kappa = np.zeros((d, d))
    else:
        # Re-index κ to the subset
        idx_old = {c: i for i, c in enumerate(mp.currencies)}
        idx_new = [idx_old[c] for c in new_currencies]
        kappa = kappa[np.ix_(idx_new, idx_new)]

    return ModelParams(
        currencies=new_currencies,
        ref_ccy=mp.ref_ccy,
        sigma=_filt(mp.sigma),
        corr=new_corr,
        k=_filt(mp.k),
        mu=_filt(mp.mu),
        gamma=mp.gamma,
        kappa=kappa,
        T_days=mp.T_days,
        pairs=new_pairs,
    )



## 2) Logistic intensity + optimal quotes

For each tier and currency-pair direction \((i,j)\), the paper uses:

\[
\Lambda(z,\delta)=\lambda(z)\,f(z,\delta),\qquad
f(z,\delta)=\frac{1}{1+\exp(\alpha(z)+\beta(z)\delta)}.
\]

The key object for pricing is the Hamiltonian (Eq. (1)):

\[
H(z,p)=\sup_\delta f(z,\delta)\,(\delta-p).
\]

The optimal quote \(\bar\delta(z,p)\) is the argmax in that supremum.
The paper notes that in the logistic case \(\bar\delta\) is easy to compute numerically (p.5).
Below we use a **robust bracket + bisection** method (no fragile Newton steps).


In [ ]:

def logistic_f(delta: float, alpha: float, beta: float) -> float:
    return 1.0 / (1.0 + np.exp(alpha + beta * delta))


def _g_prime(delta: float, p: float, alpha: float, beta: float) -> float:
    """Derivative of g(delta)=f(delta)*(delta-p) wrt delta."""
    f = logistic_f(delta, alpha, beta)
    fp = -beta * f * (1.0 - f)
    return fp * (delta - p) + f


def optimal_delta_logistic(p: float, alpha: float, beta: float,
                           tol: float = 1e-14,
                           max_iter: int = 200) -> float:
    """Compute δ̄(p)=argmax_δ f(δ)*(δ-p) for logistic f.

    Strategy:
    - show that δ̄ >= p (otherwise δ-p < 0 and payoff can't be maximal).
    - find an upper bracket where g'(δ) <= 0
    - bisection on g'(δ)=0 inside the bracket.
    """
    lower = float(p)
    # Start with a scale ~ 1/beta (natural width of the logistic)
    step = 1.0 / max(abs(beta), 1.0)
    upper = lower + step

    # Expand until derivative flips sign
    gp_upper = _g_prime(upper, p, alpha, beta)
    it = 0
    while gp_upper > 0.0 and it < 100:
        step *= 2.0
        upper = lower + step
        gp_upper = _g_prime(upper, p, alpha, beta)
        it += 1

    # Bisection
    for _ in range(max_iter):
        mid = 0.5 * (lower + upper)
        gp_mid = _g_prime(mid, p, alpha, beta)
        if gp_mid > 0.0:
            lower = mid
        else:
            upper = mid
        if (upper - lower) < tol:
            break

    return 0.5 * (lower + upper)


def H_logistic(p: float, alpha: float, beta: float) -> Tuple[float, float, float]:
    """Return (H(p), δ̄(p), f(δ̄(p)))."""
    delta_star = optimal_delta_logistic(p, alpha, beta)
    f_star = logistic_f(delta_star, alpha, beta)
    H = f_star * (delta_star - p)
    return H, delta_star, f_star



## 3) Quadratic approximation coefficients \(\alpha_0,\alpha_1,\alpha_2\)

The approximation (p.5) is:
\[
\check H(z,p)=\alpha_0(z)+\alpha_1(z)p+\tfrac12\alpha_2(z)p^2
\]
with the “natural choice”:
\[
\alpha_0(z)=H(z,0),\qquad \alpha_1(z)=\partial_p H(z,0),\qquad \alpha_2(z)=\partial_{pp}^2 H(z,0).
\]

For numerical stability:
- We use the envelope theorem: \(\partial_p H(z,p)=-f(\bar\delta(z,p))\).
- Then \(\alpha_2\) can be approximated by a centered finite difference of \(\partial_p H\) around 0.

Because the paper's example uses **discrete sizes** \(z\in\{1,5,10,20,50\}\) M$ (Table 1, p.7),
we implement sums instead of integrals.


In [ ]:

def quadratic_coeffs_H_logistic(alpha: float, beta: float,
                                eps_p: float = 1e-8) -> Tuple[float, float, float]:
    """Compute (alpha0, alpha1, alpha2) for H(p) around p=0.

    - alpha0 = H(0)
    - alpha1 = H'(0) = -f(δ̄(0))
    - alpha2 = H''(0) ≈ (H'(eps) - H'(-eps)) / (2 eps), and H'(p) = -f(δ̄(p))
    """
    H0, delta0, f0 = H_logistic(0.0, alpha, beta)
    alpha0 = H0
    alpha1 = -f0

    # H'(p) = -f(δ̄(p))
    _, _, f_plus = H_logistic(+eps_p, alpha, beta)
    _, _, f_minus = H_logistic(-eps_p, alpha, beta)
    Hp_plus = -f_plus
    Hp_minus = -f_minus
    alpha2 = (Hp_plus - Hp_minus) / (2.0 * eps_p)

    return alpha0, alpha1, alpha2



## 4) Build \(\Sigma\), \(M\), \(\tilde M\), \(P\) and solve the Riccati-like ODE (Eq. (5))

### 4.1 Covariance \(\Sigma\)

The paper uses \(\Sigma=(\rho_{i,j}\sigma_i\sigma_j)\) (p.4), where:
- \(\sigma_i\) is the volatility of currency \(i\) vs the reference currency (here USD),
- \(\rho_{i,j}\) is the correlation between those returns.

### 4.2 Discrete-size versions of \(M,\tilde M,P\)

The paper defines (p.6) matrices (here implemented as **sums over sizes**):
\[
M_{i,j}=\sum_{n}\sum_z \alpha^{n,i,j}_2(z)\,z\,\lambda^{n,i,j}(z),
\quad
\tilde M_{i,j}=\sum_{n}\sum_z \alpha^{n,i,j}_1(z)\,z\,\lambda^{n,i,j}(z),
\]
\[
P_{i,j}=\sum_{n}\sum_z \alpha^{n,i,j}_2(z)\,z^2\,\lambda^{n,i,j}(z).
\]

### 4.3 The ODE

We implement Eq. (5) as written in the paper, and integrate **backward** from \(t=T\) to \(t=0\) using an explicit Euler scheme.

> Practical note: for the paper parameters and \(T=0.05\) days, Euler with a few thousand steps is stable when the units are handled consistently (bps converted to decimals).


In [ ]:

def build_Sigma(mp: ModelParams) -> np.ndarray:
    """Build Σ from volatilities and correlations (decimal units)."""
    ccy = mp.currencies
    d = len(ccy)

    sig = np.array([mp.sigma.get(c, 0.0) for c in ccy], dtype=float)
    Sigma = np.zeros((d, d), dtype=float)

    for i in range(d):
        for j in range(d):
            if i == j:
                Sigma[i, i] = sig[i] * sig[i]
            else:
                ci, cj = ccy[i], ccy[j]
                if ci == mp.ref_ccy or cj == mp.ref_ccy:
                    rho = 0.0
                else:
                    rho = mp.corr.get(_canon_pair(ci, cj), 0.0)
                Sigma[i, j] = rho * sig[i] * sig[j]
    return Sigma


def build_M_tildeM_P(mp: ModelParams,
                     eps_p: float = 1e-8) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """Build (M, Mtilde, P) as dxd matrices.

    Convention:
    - We build entries for *ordered* pairs (i,j), i != j.
    - We assume the client-side parameters are the same for both directions inside an unordered pair.
      (This matches how Table 1 is presented in the paper.)
    """
    ccy = mp.currencies
    d = len(ccy)

    M = np.zeros((d, d), dtype=float)
    Mtilde = np.zeros((d, d), dtype=float)
    P = np.zeros((d, d), dtype=float)

    # Cache coeffs per unordered pair and tier (since they don't depend on direction)
    coeff_cache: Dict[Tuple[str, str], List[Tuple[float, float, float]]] = {}

    for i in range(d):
        for j in range(d):
            if i == j:
                continue
            key = _canon_pair(ccy[i], ccy[j])
            if key not in mp.pairs:
                continue

            pp = mp.pairs[key]

            if key not in coeff_cache:
                coeffs_per_tier = []
                for tier in pp.tiers:
                    coeffs_per_tier.append(quadratic_coeffs_H_logistic(tier.alpha, tier.beta, eps_p=eps_p))
                coeff_cache[key] = coeffs_per_tier

            z = pp.sizes_musd
            lam = pp.lambdas_per_day

            # sum over tiers and sizes
            for (a0, a1, a2) in coeff_cache[key]:
                M[i, j] += np.sum(a2 * z * lam)
                Mtilde[i, j] += np.sum(a1 * z * lam)
                P[i, j] += np.sum(a2 * (z ** 2) * lam)

    return M, Mtilde, P


def _V_of_A(A: np.ndarray, P: np.ndarray) -> np.ndarray:
    """(A) matrix as on p.6: V(A)=D(A)P + P D(A) - 2(P ⊙ A)."""
    DA = np.diag(np.diag(A))
    return DA @ P + P @ DA - 2.0 * (P * A)


def solve_AB_euler(mp: ModelParams,
                   M: np.ndarray,
                   Mtilde: np.ndarray,
                   P: np.ndarray,
                   Sigma: np.ndarray,
                   n_steps: int = 2000) -> Tuple[np.ndarray, np.ndarray]:
    """Solve Eq. (5) backward with explicit Euler.

    Returns:
        A0, B0 at time t=0.
    """
    ccy = mp.currencies
    d = len(ccy)
    U = np.ones(d)

    # Definitions on p.6
    symM = M + M.T
    M_big = np.diag(symM @ U) - symM
    V = (Mtilde - Mtilde.T) @ U

    T = mp.T_days
    dt = T / n_steps

    kappa = mp.kappa if mp.kappa is not None else np.zeros((d, d))
    A = kappa.copy()
    B = np.zeros(d)

    mu_vec = np.array([mp.mu.get(c, 0.0) for c in ccy], dtype=float)
    Dmu = np.diag(mu_vec)

    for _ in range(n_steps):
        Vtilde = (_V_of_A(A, P) - _V_of_A(A, P).T) @ U  # = 0 in symmetric examples

        A_dot = 2.0 * A @ M_big @ A - (Sigma * A) - 2.0 * Dmu @ A - 0.5 * mp.gamma * Sigma
        B_dot = mu_vec - Dmu @ B + 2.0 * A @ V + 2.0 * A @ Vtilde + 2.0 * A @ M_big @ B

        # backward integration: A(t-dt) = A(t) - dt*A_dot(t)
        A = A - dt * A_dot
        B = B - dt * B_dot

        # keep symmetry numerically
        A = 0.5 * (A + A.T)

    return A, B



## 5) Policy functions: quotes and hedging rates

### 5.1 Quotes (p.6, just above “Numerical results and discussion”)

Using \(A(t),B(t)\), the paper writes (p.6):

\[
\check\delta_{n,i,j}(t,z)=\bar\delta_{n,i,j}\!\left(z,\ \Big((2Y_{t^-}+z(e^i-e^j))^\top A(t)+B(t)^\top\Big)(e^i-e^j)\right).
\]

We implement the scalar:
\[
p_{n,i,j}(t,z,Y)=\Big((2Y+z(e^i-e^j))^\top A+B^\top\Big)(e^i-e^j),
\]
then call `optimal_delta_logistic(p, alpha, beta)`.

### 5.2 Hedging (p.6)

The paper also states (p.6):

\[
\check\xi^{i,j}_t = H^{i,j\,\prime}\!\left(
-(A(t)Y_{t^-}+B(t))^\top(e^i-e^j)
+ \frac{k^iY^i_{t^-}}{1-(A(t)Y_{t^-}+B(t))^\top e^i}
- \frac{k^jY^j_{t^-}}{1-(A(t)Y_{t^-}+B(t))^\top e^j}
\right).
\]

We implement \(H'(p)\) for the D2D execution cost
\(L(\xi)=\psi|\xi|+\eta|\xi|^2\):
\[
H'(p)=\begin{cases}
0 & |p|\le \psi \\
\frac{p-\psi\,\mathrm{sign}(p)}{2\eta} & |p|>\psi.
\end{cases}
\]

> Units:
> - inventories \(Y\) and sizes \(z\) are in **M$**,
> - time is in **days**,
> - costs and markups are in **decimals**.
>   For plotting in bps, multiply by \(10^4\).


In [ ]:

def quote_p_scalar(y: np.ndarray, i: int, j: int, z: float,
                   A: np.ndarray, B: np.ndarray) -> float:
    """Compute the 'p' input used by δ̄ in the paper's quote formula (p.6)."""
    d = y.shape[0]
    dvec = np.zeros(d)
    dvec[i] = 1.0
    dvec[j] = -1.0
    return ((2.0 * y + z * dvec) @ A + B) @ dvec


def optimal_client_markup(mp: ModelParams, tier_idx: int,
                          ccy_pay: str, ccy_sell: str,
                          z_musd: float,
                          y: np.ndarray,
                          A: np.ndarray, B: np.ndarray) -> float:
    """Compute δ_{tier, pay, sell}(z) for the client trade:
    dealer sells `ccy_sell` and receives `ccy_pay`.

    Returns δ in *decimal* (multiply by 1e4 to get bps).
    """
    ccy = mp.currencies
    i = ccy.index(ccy_pay)
    j = ccy.index(ccy_sell)

    key = _canon_pair(ccy_pay, ccy_sell)
    if key not in mp.pairs:
        return 0.0  # not traded

    pp = mp.pairs[key]
    tier = pp.tiers[tier_idx]

    p = quote_p_scalar(y, i, j, z_musd, A, B)
    return optimal_delta_logistic(p, tier.alpha, tier.beta)


def Hprime_execution_cost(p: float, psi: float, eta: float) -> float:
    """H'(p) for L(ξ)=psi|ξ|+eta ξ^2."""
    if p > psi:
        return (p - psi) / (2.0 * eta)
    if p < -psi:
        return (p + psi) / (2.0 * eta)
    return 0.0


def optimal_hedge_rate(mp: ModelParams,
                       ccy_buy: str, ccy_sell: str,
                       y: np.ndarray,
                       A: np.ndarray, B: np.ndarray) -> float:
    """Compute the paper's approximate optimal hedging rate ξ (p.6).

    Interpretation:
    - returns ξ in M$/day
    - positive ξ means: buy `ccy_buy` and sell `ccy_sell`.
    """
    ccy = mp.currencies
    i = ccy.index(ccy_buy)
    j = ccy.index(ccy_sell)

    key = _canon_pair(ccy_buy, ccy_sell)
    if key not in mp.pairs:
        return 0.0

    pp = mp.pairs[key]

    d = len(ccy)
    dvec = np.zeros(d); dvec[i] = 1.0; dvec[j] = -1.0

    AyB = A @ y + B
    base = -(AyB @ dvec)

    denom_i = 1.0 - AyB[i]
    denom_j = 1.0 - AyB[j]

    k_i = mp.k.get(ccy_buy, 0.0)
    k_j = mp.k.get(ccy_sell, 0.0)

    impact = 0.0
    if abs(denom_i) > 1e-14:
        impact += k_i * y[i] / denom_i
    if abs(denom_j) > 1e-14:
        impact -= k_j * y[j] / denom_j

    p_arg = base + impact
    return Hprime_execution_cost(p_arg, pp.psi, pp.eta)



## 6) Main wrapper: `run_multicurrency_mm(...)`

This is the main function you’ll typically call:
- builds \(\Sigma, M, \tilde M, P\)
- solves for \(A(0),B(0)\)
- returns a lightweight object containing all results plus helper callables.

You can change:
- number of currencies (by editing `mp.currencies` and `mp.pairs` or via `restrict_currencies`)
- risk aversion `mp.gamma`
- horizon `mp.T_days`
- Euler steps `n_steps`
- client parameters per pair/tier
- hedging costs per pair


In [ ]:

@dataclass
class MMResult:
    mp: ModelParams
    Sigma: np.ndarray
    M: np.ndarray
    Mtilde: np.ndarray
    P: np.ndarray
    A0: np.ndarray
    B0: np.ndarray

    def markup(self, tier_idx: int, ccy_pay: str, ccy_sell: str, z_musd: float, y: np.ndarray) -> float:
        return optimal_client_markup(self.mp, tier_idx, ccy_pay, ccy_sell, z_musd, y, self.A0, self.B0)

    def hedge_rate(self, ccy_buy: str, ccy_sell: str, y: np.ndarray) -> float:
        return optimal_hedge_rate(self.mp, ccy_buy, ccy_sell, y, self.A0, self.B0)


def run_multicurrency_mm(mp: ModelParams,
                         eps_p: float = 1e-8,
                         n_steps: int = 2000) -> MMResult:
    Sigma = build_Sigma(mp)
    M, Mtilde, P = build_M_tildeM_P(mp, eps_p=eps_p)
    A0, B0 = solve_AB_euler(mp, M, Mtilde, P, Sigma, n_steps=n_steps)
    return MMResult(mp=mp, Sigma=Sigma, M=M, Mtilde=Mtilde, P=P, A0=A0, B0=B0)



## 7) Reproduce key plots from the paper (Figures 1 and 2)

### Figure 1 (p.7): top-of-book pricing vs GBP inventory

The paper plots, for tier 1 and size 1M$:
- GBPUSD
- EURUSD
- EURGBP  
as functions of **GBP inventory**, with all other inventories set to 0.

Their caption indicates they plot:
- \( \delta_{1,X,Y} \) and \( -\delta_{1,Y,X} \) for each pair \(XY\).

Below we implement a helper that, given an FX pair name `XY`, interprets:
- **bid curve** = \( \delta_{X,Y} \) (dealer sells Y, receives X)
- **ask curve** = \(-\delta_{Y,X}\) (dealer sells X, receives Y, shown with a minus sign)

This matches how bid/ask are displayed around 0 in their figure.


In [ ]:

def _y_vector(mp: ModelParams, overrides_musd: Dict[str, float]) -> np.ndarray:
    y = np.zeros(len(mp.currencies))
    for c, v in overrides_musd.items():
        y[mp.currencies.index(c)] = v
    return y


def plot_top_of_book_quotes_vs_inventory(res: MMResult,
                                         tier_idx: int,
                                         z_musd: float,
                                         inventory_ccy: str,
                                         inventory_grid_musd: np.ndarray,
                                         pairs_to_plot: List[Tuple[str, str]]) -> None:
    """Replicate the style of Figure 1 for a set of currency pairs.

    pairs_to_plot: list of (X,Y) meaning the FX name XY.
    We plot δ_{X,Y} and -δ_{Y,X}.
    """
    mp = res.mp

    plt.figure(figsize=(9, 5))

    for X, Y in pairs_to_plot:
        bid_bps = []
        ask_bps = []
        for inv in inventory_grid_musd:
            y = _y_vector(mp, {inventory_ccy: inv})

            # bid curve: δ_{X,Y} (dealer sells Y, receives X)
            delta_bid = res.markup(tier_idx, ccy_pay=X, ccy_sell=Y, z_musd=z_musd, y=y)

            # ask curve: -δ_{Y,X} (dealer sells X, receives Y)
            delta_ask = -res.markup(tier_idx, ccy_pay=Y, ccy_sell=X, z_musd=z_musd, y=y)

            bid_bps.append(delta_bid / BP)
            ask_bps.append(delta_ask / BP)

        bid_bps = np.array(bid_bps)
        ask_bps = np.array(ask_bps)

        plt.plot(inventory_grid_musd, bid_bps, label=f"{X}{Y} bid")
        plt.plot(inventory_grid_musd, ask_bps, linestyle="--", label=f"{X}{Y} ask")

    plt.axhline(0.0, linewidth=0.8)
    plt.xlabel(f"{inventory_ccy} Inventory (M$)")
    plt.ylabel("Bid and Ask Quotes (bps)")
    plt.title("Approximate Optimal Top-of-Book Quotes")
    plt.legend(ncol=2)
    plt.grid(True, alpha=0.3)
    plt.show()



### Figure 2 (p.8): optimal externalization / hedging rates vs EUR inventory

The paper shows hedging rates for many pairs as a function of **EUR inventory** (others = 0).
Below we compute \(\check\xi\) for each pair and plot in **M$/s** for comparability with the paper's axis label.


In [ ]:

def plot_hedge_rates_vs_inventory(res: MMResult,
                                 inventory_ccy: str,
                                 inventory_grid_musd: np.ndarray,
                                 ordered_pairs_to_plot: List[Tuple[str, str]]) -> None:
    """Plot ξ (M$/s) vs inventory for selected ordered currency directions (buy, sell)."""
    mp = res.mp

    plt.figure(figsize=(9, 5))

    for buy_ccy, sell_ccy in ordered_pairs_to_plot:
        rates = []
        for inv in inventory_grid_musd:
            y = _y_vector(mp, {inventory_ccy: inv})
            xi_per_day = res.hedge_rate(buy_ccy, sell_ccy, y=y)
            rates.append(xi_per_day / DAY_SECONDS)  # -> per second

        plt.plot(inventory_grid_musd, np.array(rates), label=f"{buy_ccy}{sell_ccy}")

    plt.axhline(0.0, linewidth=0.8)
    plt.xlabel(f"{inventory_ccy} Inventory (M$)")
    plt.ylabel("Execution Rate (M$ / s)")
    plt.title("Approximate Optimal Externalization (Hedging) Rates")
    plt.legend(ncol=2)
    plt.grid(True, alpha=0.3)
    plt.show()



## 8) Run the paper example

This cell:
- builds the Table 1 parameter set,
- solves the Riccati ODE,
- reproduces plots similar to Figures 1 and 2.


In [ ]:

# --- Build the paper's example (Table 1) and solve ---
mp = build_paper_example_params()

# If you want fewer currencies, uncomment e.g.:
# mp = restrict_currencies(mp, ["USD", "EUR", "GBP"])

res = run_multicurrency_mm(mp, eps_p=1e-8, n_steps=2000)

print("Currencies:", res.mp.currencies)
print("Max |A0|:", np.max(np.abs(res.A0)))
print("Max |B0|:", np.max(np.abs(res.B0)))


In [ ]:

# --- Figure 1 style plot (p.7) ---
inv_grid = np.linspace(-100, 100, 401)  # M$
plot_top_of_book_quotes_vs_inventory(
    res=res,
    tier_idx=0,            # tier 1
    z_musd=1.0,            # 1M$ top of book
    inventory_ccy="GBP",
    inventory_grid_musd=inv_grid,
    pairs_to_plot=[("GBP", "USD"), ("EUR", "USD"), ("EUR", "GBP")],
)


In [ ]:

# --- Figure 2 style plot (p.8) ---
inv_grid = np.linspace(-100, 100, 401)  # M$

# We choose a representative list of ordered directions (buy, sell).
# You can add/remove pairs freely.
ordered_pairs = [
    ("EUR", "USD"), ("GBP", "USD"), ("CHF", "USD"), ("JPY", "USD"),
    ("EUR", "GBP"), ("EUR", "CHF"), ("EUR", "JPY"),
    ("GBP", "CHF"), ("GBP", "JPY"),
    ("CHF", "JPY"),
]

plot_hedge_rates_vs_inventory(
    res=res,
    inventory_ccy="EUR",
    inventory_grid_musd=inv_grid,
    ordered_pairs_to_plot=ordered_pairs,
)



## 9) Notes / sanity checks

A few useful diagnostics you can run:

- Check \(\Sigma\succeq 0\) (PSD).
- Inspect whether \(A_0\) is PSD (often it is, numerically).
- Check that \(B_0\approx 0\) for symmetric parameter sets (as in Table 1).

You can also increase `n_steps` if you suspect Euler discretization error.


In [ ]:

# --- Optional diagnostics ---
eig_Sigma = np.linalg.eigvalsh(res.Sigma)
eig_A0 = np.linalg.eigvalsh(res.A0)

print("Sigma eigenvalues (min/max):", eig_Sigma.min(), eig_Sigma.max())
print("A0 eigenvalues (min/max):   ", eig_A0.min(), eig_A0.max())
print("B0:", res.B0)



## 10) (Optional) Skeleton for Monte Carlo simulation

The paper also shows Monte Carlo-based inventory distributions and autocorrelations (Figures 4–5).
Implementing a faithful simulator requires careful treatment of:
- the discrete-size Poisson flows per tier and per direction,
- inventory updates \(Y\leftarrow Y \pm z\),
- hedging as continuous controls \(\xi(t)\),
- and (optionally) the price-driven diffusion of \(Y\) via \(S\).

Below is a **minimal, modular skeleton** you can extend.  
It uses a *tau-leaping* Poisson approximation at each time step `dt_sec`.


In [ ]:

def client_trade_intensity_per_sec(mp: ModelParams,
                                  tier_idx: int,
                                  ccy_pay: str, ccy_sell: str,
                                  z_musd: float,
                                  delta: float) -> float:
    """Return intensity (1/sec) for a given tier, direction, and size.

    intensity per day = λ(z) * f(δ)
    convert to per sec.
    """
    key = _canon_pair(ccy_pay, ccy_sell)
    pp = mp.pairs[key]
    tier = pp.tiers[tier_idx]

    # find the size index
    k = int(np.where(np.isclose(pp.sizes_musd, z_musd))[0][0])
    lam_day = pp.lambdas_per_day[k]

    f = logistic_f(delta, tier.alpha, tier.beta)
    return (lam_day * f) / DAY_SECONDS


def simulate_inventory_path_tau_leap(
    res: MMResult,
    T_sec: float,
    dt_sec: float,
    seed: int = 123,
    include_hedging: bool = True,
    tiers_to_use: Optional[List[int]] = None,
) -> Tuple[np.ndarray, np.ndarray]:
    """Simulate inventories Y_t (M$) under the approximate policy.

    This *simplified* simulator:
    - models client trades as independent Poisson counts per (tier, direction, size) each step
    - updates Y by +/- z for each occurrence
    - applies hedging as deterministic drift over dt (using ξ from the paper formula)

    Returns:
      times (sec), Y_path shape (n_steps+1, d)
    """
    mp = res.mp
    rng = np.random.default_rng(seed)

    d = len(mp.currencies)
    n_steps = int(np.ceil(T_sec / dt_sec))
    times = np.linspace(0.0, n_steps * dt_sec, n_steps + 1)

    Y = np.zeros((n_steps + 1, d), dtype=float)

    if tiers_to_use is None:
        # use all tiers in the config
        tiers_to_use = list(range(len(next(iter(mp.pairs.values())).tiers)))

    # Pre-list all client-tradable unordered pairs and sizes
    unordered_pairs = list(mp.pairs.keys())

    for t in range(n_steps):
        y = Y[t].copy()

        # --- client flow: iterate over unordered pairs, both directions ---
        for (a, b) in unordered_pairs:
            pp = mp.pairs[(a, b)]
            for z in pp.sizes_musd:
                for tier_idx in tiers_to_use:
                    # direction 1: pay a, sell b  (dealer sells b, receives a)
                    delta_ab = res.markup(tier_idx, ccy_pay=a, ccy_sell=b, z_musd=z, y=y)
                    lam_ab = client_trade_intensity_per_sec(mp, tier_idx, a, b, z, delta_ab)

                    # direction 2: pay b, sell a
                    delta_ba = res.markup(tier_idx, ccy_pay=b, ccy_sell=a, z_musd=z, y=y)
                    lam_ba = client_trade_intensity_per_sec(mp, tier_idx, b, a, z, delta_ba)

                    # Poisson counts over dt
                    n_ab = rng.poisson(lam_ab * dt_sec)
                    n_ba = rng.poisson(lam_ba * dt_sec)

                    if n_ab > 0:
                        ia = mp.currencies.index(a)
                        ib = mp.currencies.index(b)
                        y[ia] += n_ab * z
                        y[ib] -= n_ab * z

                    if n_ba > 0:
                        ia = mp.currencies.index(b)
                        ib = mp.currencies.index(a)
                        y[ia] += n_ba * z
                        y[ib] -= n_ba * z

        # --- hedging control (continuous in time; discretized here) ---
        if include_hedging:
            for (a, b) in unordered_pairs:
                # treat "buy a, sell b" as the positive direction
                xi_day = res.hedge_rate(a, b, y=y)  # M$/day
                xi_sec = xi_day / DAY_SECONDS
                ia = mp.currencies.index(a)
                ib = mp.currencies.index(b)
                y[ia] += xi_sec * dt_sec
                y[ib] -= xi_sec * dt_sec

        Y[t + 1] = y

    return times, Y


def autocorr(x: np.ndarray, max_lag: int) -> np.ndarray:
    """Simple autocorrelation function for 1D series."""
    x = x - np.mean(x)
    denom = np.dot(x, x)
    if denom <= 0:
        return np.zeros(max_lag + 1)
    ac = np.empty(max_lag + 1)
    ac[0] = 1.0
    for k in range(1, max_lag + 1):
        ac[k] = np.dot(x[:-k], x[k:]) / denom
    return ac



Example usage (short run for speed):

> To approach Figures 4–5 in the paper, you need long runs (millions of seconds) and careful tuning of dt.


In [ ]:

# --- quick demo simulation (keep short unless you have time) ---
T_demo_sec = 60.0 * 30.0   # 30 minutes
dt_sec = 0.25              # 250 ms

times, Ypath = simulate_inventory_path_tau_leap(
    res,
    T_sec=T_demo_sec,
    dt_sec=dt_sec,
    seed=42,
    include_hedging=True,
)

# Plot EUR inventory path as a sanity check
eur_idx = res.mp.currencies.index("EUR")
plt.figure(figsize=(9, 3))
plt.plot(times/60.0, Ypath[:, eur_idx])
plt.xlabel("Time (min)")
plt.ylabel("EUR inventory (M$)")
plt.title("Demo simulated EUR inventory path (tau-leap)")
plt.grid(True, alpha=0.3)
plt.show()
